# Wk07 (3/9)

## 책

- Ch7 Photo Mosaic
- Ch8 Auto Streosystem

## Python Library
- PIL
- NumPy

## Python 문법
- 예외처리 (try / except)
- assert
- 파일 처리하기 : 많은 이미지 파일 처리하기!

## Ch7 Photo Mosaic

- 수많은 사각형들로 분할된 이미지, 각 사각형은 타깃과 일치하는 다른 이미지로 대체된다. 
- https://en.wikipedia.org/wiki/Photographic_mosaic

### 동작 원리 (알고리즘)

1. 타깃 이미지 내의 타일을 대체할 입력 이미지들을 읽어 들인다. 
2. 타깃 이미지를 읽어들인 다음 M X N 그리드로 분할한다.
3. 각 타일별로 입력 이미지들 중에서 가장 일치하는 이미지를 찾는다.
4. 선택된 입력 이미지들을 M X N 그리드에 배열해 최종 모자이크를 생성한다.

#### Step1 타일 이미지 읽기

In [2]:
def getImages(imageDir):
    files = os.listdir(imageDir)
    images = []
    for file in files:
        filePath = os.path.abspath(os.path.join(imageDir, file))
        try:
            fp = open(filePath, "rb")
            im = Image.open(fp)
            images.append(im)
            im.load()
            fp.close()
        except:
            print("Invalid image: %s" %(filePath))
    return images

- os.listdir() : imageDir 디렉토리 내의 파일들을 리스트로 읽어들인다. 
- os.path.abspath()와 os.path.join()함수를 사용해 이미지의 파일명을 얻어온다. (상대경로와 절대경로에 모두 동작 가능)
- 리스트 내의 파일을 PIL의 image객체로 불러들인다. 
- 파일을 일일이 객체로 불러오기 위해 파일명을 일일이 Image.open()메소드로 전달할 수도 있지만, 폴더 내의 이미지의 개수가 수백 수천개에 이르기 때문에 시스템의 자원을 너무 많이 사용한다. 
- 타일 이미지들을 연 다음에, 파일을 닫고 시스템 자원을 해제한다. 
    - oepn()함수로 이미지 파일을 연다
    - 결과 이미지 im을 배열에 저장한다.

#### Step2: 이미지 평균 색상 값 계산하기

In [4]:
def getAverageRGB(image):
    im = np.array(image)
    w,h,d = im.shape
    return tuple(np.average(im.reshape(w*h, d), axis=0))

#### Step3: 타깃 이미지 분할하기

In [5]:
def splitImage(image, size):
    W, H = image.size[0], image.size[1]
    m, n = size
    w, h = int(W/n), int(H/m)
    imgs = []
    for j in range(m):
        for i in range(n):
            imgs.append(image.crop((i*w, j*h, (i+1)*w, (j+1)*h)))
    return imgs

#### Step4: 타일과 가장 비슷한 이미지 찾기

In [6]:
def getBestMatchIndex(input_avg, avgs):
    avg = input_avg
    index = 0
    min_index = 0
    min_dist = float("inf")
    for val in avgs:
        dist = ((val[0] - avg[0])**2 + (val[1] - avg[1])**2 + (val[2] - avg[2])**2)
        if dist < min_dist:
            min_dist = dist
            min_index = index
        index += 1
    return min_index 

- avgs 리스트에서 평균 RGB값 input_avg와 가장 근접하는 것을 찾으려고 시도한다. 
- avgs 리스트는 입력 이미지들의 입력 RGB값이 들어 있는 리스트다. 
- 가장 비슷한 것을 찾기 위해 입력 이미지들의 평균 RGB값과 비교가 수행된다. 

#### Step5 이미지 그리드 생성하기

In [7]:
def createImageGrid(images, dims):
    m, n = dims
    assert m*n == len(images)

    width = max([img.size[0] for img in images])
    height = max([img.sizep[1] for img in images])

    grid_img = Image.new('RGB', (n*width, m*height))

    for index in range(len(images)):
        row = int(index/n)
        col = index - n*row
        grid_img.paste(images[index], (col*width, row*height))
    
    return grid_img

- 그리드 크기를 수집한 다음, assert를 사용해 createImageGrid()에 전달된 이미지의 개수가 그리드 크기와 일치하는지 확인한다. (assert 메소드는 특해 개발 과정에서 코드 내의 가정을 확인할 때 유용하다.)
- 지금 가장 비슷한 RGB값에 기반해 타일 이미지 리스트가 주어져 있으며, 이 리스트를 사용해 포토 모자이크를 나타내는 이미지 그리드가 생성된다. 

#### Step6 포토모자이크 생성하기 

In [8]:
def createPhotomosaic(target_image, input_images, grid_size, reuse_image=True):
    print("splitting input image...")
    target_images = splitImage(target_image, grid_size)
    print('finding image matches...')
    output_images = []
    count = 0
    batch_size = int(len(target_images)/10)

    avgs = []
    for img in input_images:
        avgs.append(getAverageRGB(img))
        match_index = getBestMatchIndex(avg, avgs)
        output_images.append(input_images[match_index])
        if count > 0 and batch_size > 10 and count % batch_size is 0:
            print('processed %d of %d...' %(count, len(target_images)))
        count += 1
        if not reuse_images:
            input_images.remove(match)
        
    print('creating mosaic...')
    mosaic_image = createImageGrid(output_images, grid_size)
        
    return mosaic_image

- 타깃 이미지, 입력 이미지들의 리스트, 생성될 포토모자이크의 크기, 이미지 재사용 기능 여부를 나타내는 플래그를 인수로 받는다. 

#### Step7: 포토모자이크 크기 제어하기

In [10]:
print('resizing image...')
#dims = (int(target_image.size[0]/grid_size[1]),
#        int(target_image.size[1]/grid_size[0]))
#print('max tile dims: %s' %(dims))
#for img in input_images:
#    img.thumbnail(dims)

resizing image...


### 전체 코드

In [12]:
def getAverageRGB(image):
    #step2: 이미지 평균 색상값 계산하기
    ''' PIL Image 객체가 주어졌을 때, 색상의 평균값을 (r, g, b) 형태로 반환한다. '''
    pass

def splitImage(image, size):
    #Step3: 타깃 이미지 분할하기
    ''' Image 객체와 크기 정보가 주어졌을 때, (m*n)크기의 Image리스트를 반환한다. '''
    pass

def getImage(imageDir):
    #step1: 타일 이미지 읽기 
    '''디렉토리를 인수로 받아서 이미지들의 리스트를 반환한다. '''
    pass

def getImageFilenames(imageDir):
    #step1-1: 이미지들이 위치하는 디렉토리가 주어졌을 때, 이미지 파일명들의 리스트를 반환한다. 
    '''이미지들이 위치하는 디렉트로가 주어졌을 때, 이미지 파일명들의 리스트를 반환한다. '''
    pass

def getBestMatchIndex(input_avg, avgs):
    #step4: 타일과 가장 비슷한 이미지 찾기
    '''평균 RGB 거리를 기준으로 가장 가까운 이미지의 인덱스를 반환한다. '''
    pass

def createImageGrid(images, dims):
    #step5: 이미지 그리드 생성하기 
    '''이미지 리스트와 그리드 크기(m, n)이 주어졌을 때, 이미지들로 이뤄진 그리드를 생성한다.'''
    pass

def createPhotomosaic(target_image, input_image, grid_size, reuse_image=True):
    #step6: 포토모자이크 생성하기 
    '''타겟 이미지와 입력 이미지들이 주어졌을 때, 포토모자이크를 생성한다. '''
    pass

def main():
    #타겟 이미지
    target_image = Image.open(args.target_image)
    #입력 이미지, getImage함수 이용
    input_images = getImage(args.input_folder)
    #그리드의 크기
    grid_size = int(args.grid_size[0]), int(args.grid_size[1])
    #출력 파일
    output_filename = 'mosaic.png'
    if args.outfile:
        output_filename = args.outfile
    #포토 모자이크 생성
    mosaic_image = createPhotomosaic(target_image, input_images, grid_size, reuse_images=True)
    #포토 모자이크 저장
    mosaic_image.save(output_filename, 'PNG')


## Ch8 Auto Stereogram

- 오토스테레오그램은 매직아이로서, 3차원적 지각을 만들어내는 2차원 이미지다. 
- 3차원처럼 보이는 반복패턴으로 구성된다. 
- 입력받은 깊이맵으로부터 오토스테레오그램을 생성한다.

## 동작원리 (알고리즘)

- 깊이맵을 읽어 들인다. 
- 타일 이미지를 읽어들이거나 무작위 점 타일을 생성한다.
- 타일을 반복해 새로운 이미지를 생성한다. 이 이미지의 폭과 높이는 깊이맵과 일치해야 한다. 
- 새로 생성된 이미지의 모든 픽셀에 대해, 그 픽셀의 깊이 값에 비례해서 오른쪽으로 픽셀을 시프트 한다. 
- 오토스테레오그램을 파일에 기록한다. 

### Step1: 주어진 타일 반복하기

In [13]:
def createTiledImage(tile, dims):
    img = Image.new('RGB', dims)
    W, H = dims
    w, h = tile.size
    cols = int(W/w) + 1
    rows = int(H/h) + 1
    for i in range(rows):
        for j in range(cols):
            img.paste(tile, (j*w, i*h))
    return img

### Step2 임의의 원으로 타일 생성하기

In [14]:
def createRandomTile(dims):
    img = Image.new('RGB', dims)
    draw = ImageDraw.Draw(img)
    r = int(min(*dims)/100)
    n = 1000
    for i in range(n):
        x, y = random.randint(0, dims[0]-r), random.randint(0, dims[1]-r)
        fill = (random.randint(0, 255), random.randint(0, 255), random.randtint(0,255))
        draw.ellipse((x-r, y-r, x+r, y+r), fill)
    return img

### Step3: 오토스테레오그램 만들기

In [15]:
def createAutostereogram(dmap, tile):
    if dmap.mode is not 'L':
        dmap = dmap.convert('L')
    if not tile:
        tile = createRandomTile((100, 100))
    img = createTiledImage(tile, dmap.size)
    sImg = img.copy()
    pixD = dmap.load()
    pixS = sImg.load()
    cols, rows = sImg.size
    for j in range(rows):
        for i in range(cols):
            xshift = pixD[i, j]/10
            xpos = i - tile.size[0] + xshift
            if xpos > 0 and xpos < cols:
                pixS[i, j] = pixS[xpos, j]
    return sIMG

## 전체코드

In [ ]:
def createSpacingDepthExample():
    pass

def createRandomTile(dims):
    pass

def createTiledImage(tile, dims):
    pass

def createDepthMap (dims):
    pass

def createDepthShiftedImage(dmap, img):
    pass

def createAutostereogram(dmap, tile):
    pass

def main():
    #출력 파일 설정
    outFile = 'as.png'
    if args.outFile:
        outFile = args.outFile
    #타일 설정
    tileFile = False
    if args.tileFile:
        tileFile = Image.open(args.tileFile)
    #깊이맵을 연다
    dmImg = Image.open(args.dmFile)
    #스테레오그램 생성 createAutostereogram()함수 호출
    asImg = createAutostereogram(dm(Img, tileFile))
    #파일 저장
    asImg.save(outFile)